<a href="https://colab.research.google.com/github/ammubharatram/AI-Crowd-Blitz-Challenge/blob/master/ORIENTME_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline for ORIENTME Challenge on AIcrowd
#### Author : Bharat Ram Ammu


## To open this notebook on Google Colab, click below!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/gist/aicrowd-bot/5ef00d22288220f0e87e73b74ce55a13)


# Increase **RAM** upto 26GB

In [0]:
# d=[]
# while(1):
#   d.append('1')

# To decrease the memory

In [0]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

# Check GPU 

In [3]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Download Necessary Packages

In [4]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scikit-learn 
!{sys.executable} -m pip install matplotlib tqdm 

## Download data
The first step is to download the training data and the test data


In [5]:
# #Donwload the datasets
!rm -rf data/
!mkdir data/

!wget https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/orientme/v0.1/training.tar.gz -O data/training.tar.gz
!wget https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/orientme/v0.1/test.tar.gz -O data/test.tar.gz
!wget https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/orientme/v0.1/sample_submission.csv -O data/sample_submission.csv
!tar xvzf data/training.tar.gz -C data/
!tar xvzf data/test.tar.gz -C data/

Streaming output truncated to the last 5000 lines.
images/059612.jpg
images/078804.jpg
images/092205.jpg
images/085779.jpg
images/070280.jpg
images/084467.jpg
images/050150.jpg
images/076829.jpg
images/060093.jpg
images/083308.jpg
images/069909.jpg
images/094674.jpg
images/071820.jpg
images/057159.jpg
images/077289.jpg
images/094112.jpg
images/050636.jpg
images/084301.jpg
images/092563.jpg
images/066384.jpg
images/051528.jpg
images/056247.jpg
images/082770.jpg
images/059174.jpg
images/076197.jpg
images/096705.jpg
images/099436.jpg
images/081279.jpg
images/052021.jpg
images/086516.jpg
images/074958.jpg
images/089625.jpg
images/064593.jpg
images/087608.jpg
images/055996.jpg
images/090374.jpg
images/074780.jpg
images/065855.jpg
images/054450.jpg
images/080167.jpg
images/098728.jpg
images/058242.jpg
images/078192.jpg
images/057171.jpg
images/083446.jpg
images/071808.jpg
images/084329.jpg
images/093655.jpg
images/068381.jpg
images/082980.jpg
images/051500.jpg
images/085037.jpg
images/060905

In [0]:
## Now the data is available at the following locations:

TRAINING_IMAGES_FOLDER = "data/training/images/"
TRAINING_LABELS_PATH = "data/training/labels.csv"
TEST_IMAGES_FOLDER = "data/images"
SAMPLE_SUBMISSION_FILE_PATH = "data/sample_submission.csv"


## Import packages

In [0]:
import os
import tqdm

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image 


## Load Data
We use PIL library to load our images. Here we are creating our array where our input features are the mean colours and output features are the rotations along the 3 axis.

In [0]:
training_labels_df = pd.read_csv(TRAINING_LABELS_PATH)

def pre_process_data_X(image):
    """
    This file takes a loaded image and returns a particular 
    representation of the data point
    
    
    NOTE: This current baseline implements a **very** silly approach
    of representing every image by the mean RGB values for every image.
    
    You are encourage to try to alternate representations of the data,
    or figure out how to learn the best representation from the data ;)
    """
    im_array = np.array(im)
    mean_rgb = im_array.mean(axis=(0, 1))
    return mean_rgb


ALL_DATA = []

for _idx, row in tqdm.tqdm(training_labels_df.iterrows(), total=training_labels_df.shape[0]):
    filepath = os.path.join(
        TRAINING_IMAGES_FOLDER,
        row.filename
    )
    im = Image.open(filepath)
    
    data_X = pre_process_data_X(im)
    data_Y = [row.xRot, row.yRot, row.zRot]
    
    ALL_DATA.append((data_X, data_Y))



 23%|██▎       | 11257/50000 [02:06<07:28, 86.46it/s]

## EDA
We now see the kind of images the dataset contains to get a better idea. The title signifies clockwise rotation of the cube along that axis

In [0]:
plt.figure(figsize=(20,20))
for i in range(16):
  path,x,y,z = training_labels_df.iloc[i]
  filepath = os.path.join(
        TRAINING_IMAGES_FOLDER,
        path
    )
  im = Image.open(filepath)
  plt.subplot(4,4,i+1)
  plt.axis('off')
  plt.title("x: %.3f, y: %.3f, z: %.3f"%(x,y,z))
  plt.imshow(im)

## Split Data into Train and Validation
We split the dataset into Training data and Validation datasets to help us test the generalizability of our models, and to ensure that we are not overfitting on the training set.

In [0]:
training_set, validation_set= train_test_split(ALL_DATA, test_size=0.2, random_state=42) 

Here we have selected the size of the testing data to be 20% of the total data. You can change it and see what effect it has on the accuracies. To learn more about the train_test_split function [click here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

Now, since we have our data splitted into train and validation sets, we need to get the label separated from the data.

In [0]:
X_train, y_train = zip(*training_set)
X_val, y_val = zip(*validation_set)


X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)

## Define the Classifier
Now we finally come to the juicy part. 
Now that all the data is all loaded and available nice, we can finally get to training the classifier. Here we use sklearn [`MLPRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html) to train our network. We can tune the hyper parameters based on cross validation scores 

In [0]:
model = MLPRegressor(hidden_layer_sizes=[10, 10], verbose=True)
# NOTE : This is again silly hyper parameter instantiation of this problem,
# and we encourage you to explore what works the best for you.

## Train the classifier

In [0]:
model.fit(X_train, y_train)

Iteration 1, loss = 16747.09323116
Iteration 2, loss = 6267.81371355
Iteration 3, loss = 5881.67650825
Iteration 4, loss = 5835.24373353
Iteration 5, loss = 5795.39806423
Iteration 6, loss = 5739.75166014
Iteration 7, loss = 5691.85701342
Iteration 8, loss = 5668.52422696
Iteration 9, loss = 5652.33722598
Iteration 10, loss = 5639.78673227
Iteration 11, loss = 5627.61985644
Iteration 12, loss = 5615.66774653
Iteration 13, loss = 5601.60722025
Iteration 14, loss = 5589.40557733
Iteration 15, loss = 5573.34865609
Iteration 16, loss = 5552.97766248
Iteration 17, loss = 5534.74783801
Iteration 18, loss = 5518.71086587
Iteration 19, loss = 5507.00724045
Iteration 20, loss = 5496.52598947
Iteration 21, loss = 5489.63235026
Iteration 22, loss = 5483.56160412
Iteration 23, loss = 5479.29583036
Iteration 24, loss = 5475.31918002
Iteration 25, loss = 5472.33014656
Iteration 26, loss = 5470.54825485
Iteration 27, loss = 5468.52853450
Iteration 28, loss = 5465.51588756
Iteration 29, loss = 5462.90

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=[10, 10], learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=True,
             warm_start=False)

## Predict on Validation
Now we predict our trained classifier on the validation set and evaluate our model

In [0]:
y_pred = model.predict(X_val)

## Evaluate the Performance
We use the same metrics as that will be used for the test set.  
[MAE](https://en.wikipedia.org/wiki/Mean_absolute_error) and [RMSE](https://www.statisticshowto.com/rmse/) are the metrics for this challenge

In [0]:
print('Mean Absolute Error:', mean_absolute_error(y_val, y_pred))  
print('Mean Squared Error:', mean_squared_error(y_val, y_pred))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_val, y_pred)))

Mean Absolute Error: 90.37427639406944
Mean Squared Error: 10872.550637486334
Root Mean Squared Error: 104.27152361736321


## Load Test Set
Load the test data now

In [0]:
import glob

TEST_DATA = []
TEST_FILENAMES = []

for _test_image_path in tqdm.tqdm(glob.glob(os.path.join(TEST_IMAGES_FOLDER, "*.jpg"))):
    filename = os.path.basename(_test_image_path)
    im = Image.open(_test_image_path)
    
    data_X = pre_process_data_X(im)
    TEST_DATA.append(data_X)
    TEST_FILENAMES.append(filename)

100%|██████████| 50001/50001 [07:41<00:00, 108.30it/s]


In [0]:
TEST_DATA = np.array(TEST_DATA)
# model = classifier

## Make predictions on the test set

In [0]:
test_predictions = model.predict(TEST_DATA)

In [0]:
test_df = pd.DataFrame(test_predictions, columns=['xRot', 'yRot', 'zRot'])
test_df["filename"] = TEST_FILENAMES

## Save the prediction to csv

In [0]:
test_df.to_csv('submission.csv', index=False)

Note: Do take a look at the submission format.The submission file should contain a header.

## To download the generated csv in Google Colab run the below command

In [0]:
from google.colab import files
files.download('submission.csv')

### Go to [platform](https://www.aicrowd.com/challenges/aicrowd-blitz-may-2020/problems/orientme). Participate in the challenge and submit the submission.csv.